In [ ]:
import pandas as pd
import spacy
from sklearn.metrics import make_scorer, classification_report, make_scorer, recall_score, f1_score, accuracy_score, confusion_matrix
df = pd.read_excel('../data/avis/general_df_clean_sent_15k_manual_code.xlsx')

nlp = spacy.load("fr_core_news_lg")
df.head()



In [ ]:
theme_frais= [row['text_avis'] for index, row in df.iterrows() if pd.notna(row['var_frais'])]
theme_efficacite = [row['text_avis'] for index, row in df.iterrows() if pd.notna(row['var_efficacité'])]
theme_discrimination = [row['text_avis'] for index, row in df.iterrows() if pd.notna(row['var_discrimination'])]
theme_protexction = [row['text_avis'] for index, row in df.iterrows() if pd.notna(row['var_protection'])]
theme_cloture = [row['text_avis'] for index, row in df.iterrows() if pd.notna(row['var_clôture'])]
theme_prets = [row['text_avis'] for index, row in df.iterrows() if pd.notna(row['var_prêts'])]
theme_virements = [row['text_avis'] for index, row in df.iterrows() if pd.notna(row['var_virements'])]
theme_communication = [row['text_avis'] for index, row in df.iterrows() if pd.notna(row['var_communication'])]


In [ ]:
nuggets_efficacité = pd.read_excel("../data/avis/general_df_clean_sent_15k_manual_code.xlsx",
                                   sheet_name = "curation_efficacité", index_col= 0)
nuggets_efficacité.head(1)

In [ ]:
# tout
list_exemples_tout = df.text_avis

# efficace 
list_exemples_efficace = nuggets_efficacité.curated_efficace
list_exemples_efficace = list_exemples_efficace.dropna()
bag_efficace = " ".join([message for message in list_exemples_efficace])

# innefficace
list_exemples_innefficace = nuggets_efficacité.curated_inneficace
list_exemples_innefficace = list_exemples_innefficace.dropna()
bag_innefficace = " ".join([message for message in list_exemples_innefficace])
bag_innefficace_nlp =nlp(bag_innefficace)


In [ ]:
# fonctions et liste de seuils de similarité à tester

import numpy as np
test_thresh = np.arange(0.59,1.01, 0.01)
#thresh_bench = pd.DataFrame(columns=["thresh", "f1-score"])
f1_scores = []

def allocate_lab(sim_score, y_pred, sim_thresh):
    if max(sim_score) > sim_thresh:
        y_pred.append(1)
    else:
        y_pred.append(0)

def similarity_report(y,y_pred):
    report =classification_report(y, y_pred, output_dict=True)  
    macro_f1 = round(report['macro avg']['f1-score'],2)
    #print(report)
    f1_scores.append(macro_f1)


In [ ]:
# [message] to [bag of exemples] ->> f1 =0.7 avec thresh de 0.84
f1_scores = []

for thresh in test_thresh:
    y_pred = []
    for message in list_exemples_tout:
        sim_score = []
        #print("message entier : ",message)
        message = nlp(message)
        sim_score.append(round(bag_innefficace_nlp.similarity(message),2))
        #print(sim_score)
        #print("-------------")
        allocate_lab(sim_score,y_pred,thresh)
    similarity_report(y,y_pred)
best_score = max(f1_scores)
best_thresh = test_thresh[f1_scores.index(best_score)]
print(f"best score: {best_score} using similarity thresh of {best_thresh}")

In [ ]:
# [sentence of messages] vs [bag of exemples] --> score f1 : 0.73 avec thresh de 0.84
f1_scores = []

for thresh in test_thresh:
    y_pred = []
    for message in list_exemples_tout:
        sim_score= []
        #print("message entier : ",message)
        message = nlp(message)
        sentences = message.sents
        for sentence in sentences:
            #print("phrase: ",sentence)
            temp_sim_score = round(bag_innefficace_nlp.similarity(sentence),2)
            #print(temp_sim_score)
            sim_score.append(temp_sim_score)
        #print("-------------")
        allocate_lab(sim_score,y_pred,thresh)
    similarity_report(y,y_pred)

best_score = max(f1_scores)
best_thresh = test_thresh[f1_scores.index(best_score)]
print(f"best score: {best_score} using similarity thresh of {best_thresh}")

In [ ]:
# [sentence of messages] vs [étalon] --> f1 .57 avec seuil de 0.59
f1_scores = []

for thresh in test_thresh:
    y_pred = []
    for message in list_exemples_tout:
        sim_score= []
        #print("message entier : ",message)
        message = nlp(message)
        sentences = message.sents
        for sentence in sentences:
            #print("phrase: ",sentence)
            temp_sim_score = round(étalon.similarity(sentence),2)
            #print(temp_sim_score)
            sim_score.append(temp_sim_score)
        #print("-------------")
        allocate_lab(sim_score,y_pred,thresh)
    similarity_report(y,y_pred)

best_score = max(f1_scores)
best_thresh = test_thresh[f1_scores.index(best_score)]
print(f"best score: {best_score} using similarity thresh of {best_thresh}")

In [ ]:
best_score = max(f1_scores)
best_thresh = test_thresh[f1_scores.index(best_score)]
print(f"best score: {best_score} using similarity thresh of {best_thresh}")

In [ ]:
# [sentence of messages] vs [étalon] --> f1 .54 avec seuil de 0.59 
f1_scores = []

for thresh in test_thresh:
    y_pred = []
    for message in list_exemples_tout:
        sim_score = []
        #print("message entier : ",message)
        message = nlp(message)
        sim_score.append(round(étalon.similarity(message),2))
        #print(sim_score)
        #print("-------------")
        allocate_lab(sim_score,y_pred,thresh)
    similarity_report(y,y_pred)
best_score = max(f1_scores)
best_thresh = test_thresh[f1_scores.index(best_score)]
print(f"best score: {best_score} using similarity thresh of {best_thresh}")

In [ ]:
# [sentences of messages] vs [sentences of bag of exemples] -> f1-score of .84 avec seuil de 0.91
f1_scores = []

for thresh in test_thresh:
    y_pred = []
    for message in list_exemples_tout:
        sim_score= []
        #print("message entier : ",message)
        message = nlp(message)
        sentences = message.sents
        sentences_exemples = bag_innefficace_nlp.sents
        for sentence in sentences:
            for sentence_exemple in sentences_exemples:
                #print("phrase: ",sentence)
                temp_sim_score = round(sentence_exemple.similarity(sentence),2)
                #print(temp_sim_score)
                sim_score.append(temp_sim_score)
        #print("-------------")
        allocate_lab(sim_score,y_pred,thresh)
    similarity_report(y,y_pred)

best_score = max(f1_scores)
best_thresh = test_thresh[f1_scores.index(best_score)]
print(f"best score: {best_score} using similarity thresh of {best_thresh}")

In [ ]:
test= pd.read_excel("")

In [ ]:
# version sentence sans stopwords



In [ ]:
definition = "def"
exemples = "str concaténé des exemples"
defex = definition + " " + exemples

# boucle pour évaluer pour chaque theme les messages, puis les phrases dans chaque message

# boucle thèmes
for thème in df.themes:
	col_theme = "Q" + thème
	# boucle pour les messages
	for i, message in enumerate(train_df.text_total):
		message = nlp(message)
		message_sentence = sentence(message)
		sim_scores = []
		for sentence in message_sentence:
			sim = sentence.similarity(defex)
			sim_scores.append(sim)
		if sim_score.max() > 0.8:
			train_df.iloc[i,col_theme] = 1
			exerpt_col = col_theme + "exerpt"
			train_df.iloc[i,exerpt_col] = sentence[sim_scores.index(max(sim_scores))]
		else:
			train_df[col_theme] = 0
			exerpt_col = ""
 
	



In [187]:
#  TEST [sentences of messages] vs [sentences of bag of exemples] -> 0.72 using similarity thresh of 0.83
test = pd.read_excel("../data/avis/general_df_clean_sent_15k_manual_code.xlsx",
                                   sheet_name = "test", index_col= 0)
test_text = test.text_avis
y = test.var_efficacité_bad
test_messages = test.text_avis.dropna()

f1_scores = []

for thresh in test_thresh:
    y_pred = []
    for message in test_messages:
        sim_score= []
        #print("message entier : ",message)
        message = nlp(message)
        sentences = message.sents
        sentences_exemples = bag_innefficace_nlp.sents
        for sentence in sentences:
            for sentence_exemple in sentences_exemples:
                #print("phrase: ",sentence)
                temp_sim_score = round(sentence_exemple.similarity(sentence),2)
                #print(temp_sim_score)
                sim_score.append(temp_sim_score)
        #print("-------------")
        allocate_lab(sim_score,y_pred,thresh)
    similarity_report(y,y_pred)

best_score = max(f1_scores)
best_thresh = test_thresh[f1_scores.index(best_score)]
print(f"best score: {best_score} using similarity thresh of {best_thresh}")

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

best score: 0.72 using similarity thresh of 0.8300000000000002


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

In [191]:
# essai sur l'ensemble de la base d'entrainement pour la variable inefficacité

df = pd.read_csv("../data/avis/train_noYC_lemma_sent_equil.csv", index_col= 0)
df_messages = df.text_total

f1_scores = []
thresh = 0.83 # as per our grid search results

y_pred = []
for message in df_messages:
    sim_score= []
    #print("message entier : ",message)
    message = nlp(message)
    sentences = message.sents
    sentences_exemples = bag_innefficace_nlp.sents
    for sentence in sentences:
        for sentence_exemple in sentences_exemples:
            #print("phrase: ",sentence)
            temp_sim_score = round(sentence_exemple.similarity(sentence),2)
            #print(temp_sim_score)
            sim_score.append(temp_sim_score)
    #print("-------------")
    allocate_lab(sim_score,y_pred,thresh)

df['var_bad:inefficace'] = y_pred

best_score = max(f1_scores)
best_thresh = test_thresh[f1_scores.index(best_score)]
print(f"best score: {best_score} using similarity thresh of {best_thresh}")

/var/folders/7m/gxswx1js22x0hszgmzpd29qcg1657r/T/ipykernel_94625/1558276724.py:19: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  temp_sim_score = round(sentence_exemple.similarity(sentence),2)
